# Tutorial

In this document, we'll show how to use this package, and compare the results to a frequentist, ad-hoc method of estimating the diffusion coefficient and its uncertainty.

In [1]:
%matplotlib inline

import Bayesian_Particle_Tracking
from Bayesian_Particle_Tracking import model
from Bayesian_Particle_Tracking.model import log_likelihood, log_posterior, log_prior, diffusion
from Bayesian_Particle_Tracking import io
from Bayesian_Particle_Tracking.prior import JeffreysPrior, UniformPrior
from Bayesian_Particle_Tracking.printable import Printable
from Bayesian_Particle_Tracking import generate_data
from Bayesian_Particle_Tracking.generate_data import data_generation, generator
import emcee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

We will use generated data so that it will be convenient for evaluating the validity of our method, since we will know the diffusion coefficient. For our parameters we choose $\sigma=10^{-8}$m, $\mu=10^{-4}$Pa\*s, $a=10^{-8}$m, $\tau = 1$s, where $\sigma$ is the measurement uncertainty on the particle's position, $\mu$ is the dynamic viscosity of the medium, $a$ is the radius of the particle, and $\tau$ is the time constant. the Our starting position is at the origin. We will look at a single particle over 10000 time frames or steps.

Note that from our parameters we are yielded a value of D by Stokes-Einstein:
$$D=\frac{k_bT}{6\pi\mu a}=2.1973*10^{-10} m^2/s$$

In [2]:
#Generated the data with the following code.
#data = Bayesian_Particle_Tracking.generate_data.generator(100000,10**(-8),10**(-4),10**(-8),[0,0,0])
#np.save('compare_data', data)

home_dir = "/Users/alanzhou/Documents/Physics_201/final_project/"
data = np.load(home_dir + 'compare_data.npy')
compare_input = diffusion(data)

To evaluate the diffusion coefficient we will be using Markov Chain Monte Carlo with the emcee package.

In [ ]:
#the model has 1 parameter; we'll use 40 walkers and 500 steps
ndim = 1
nwalkers = 40
nsteps = 500

#starting_positions = [abs(10**(-10) + 1e-10*np.random.randn(ndim)) for i in range(nwalkers)]

starting_positions = emcee.utils.sample_ball(
    ([10**(-10)]),
    ([10**(-10)]),nwalkers)

# set up the sampler object
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, 
                                args=[compare_input])
# run the sampler. We use iPython's %time directive to tell us 
# how long it took (in a script, you would leave out "%time")
%time sampler.run_mcmc(starting_positions, nsteps)
print('Done')

Plotting our walkers:

In [ ]:
fig, (ax_D) = plt.subplots(1)
ax_D.set(ylabel='D')
for i in range(nwalkers):
    sns.tsplot(sampler.chain[i,:,0], ax=ax_D)

This is a pretty fast burn in time. We'll cut out the first 75 steps.

In [ ]:
samples = sampler.chain[:,75:,:]
# reshape the samples into a 1D array where the column is D
traces = samples.reshape(-1, ndim).T
# create a pandas DataFrame with labels. 
parameter_samples = pd.DataFrame({'D': traces[0]})

Now let's plot the marginal pdf of D:

In [ ]:
sns.kdeplot(parameter_samples['D'])
plt.title('$D$')

In [ ]:
q = parameter_samples.quantile([0.16,0.50,0.84], axis=0)

print("D = {:.3e} + {:.3e} - {:.3e}".format(q['D'][0.50], 
                                            q['D'][0.84]-q['D'][0.50],
                                            q['D'][0.50]-q['D'][0.16]))

This is very close to what we expect, and the true value of D is contained within our 68% credibility interval.

## Ad Hoc Method

This method is taken from Jerome Fung's 2013 doctoral dissertation at Harvard University, _Measuring the 3D Dynamics of Multiple Colloidal Particles_. The method, described as a "Flyvbjerg-Peterson (FB) algorithm" is described as follows:

1. Begin with correlated data (i.e., a set of squared displacements) $x_i$
2. Block-decorrelate the data (with Flybjerg-Peterson block-decorrelation). At each block decorrelation step, calculate var($x'_i$)/(N'-1)
3. To find the leftmost fixed-point region (smallest number of decorrelation setps), check if the variance estimate after $j$ transformations lies within the 1-sigma error bars of the variance estimate after j+1 transformations. The leftmost point to be considered fixed is the first point satisfying this criterior.
4. Repeat Step 3 from the right to find the rightmost edge of the fixed point region.
5. Compute a weighted average of all the var($x'_i$)/(N'-1) in the fixed-point region, where the weights by the 1-sigma error bars.

First we define our squared displacement function.

We'll also define a function to calculate MSD for illustrative purposes

In [ ]:
def sq_displacement(data):
    data_length = len(data)
    point_before = data[:len(data)-1]
    x_before, y_before, z_before = point_before[:,0], point_before[:,1], point_before[:,2]

    data_points = data[1:]
    x_data, y_data, z_data = data_points[:,0], data_points[:,1], data_points[:,2]
    distance = np.sqrt((x_before-x_data)**2+(y_before-y_data)**2+(z_before-z_data)**2)
    return distance**2

def MSD(data):
    return np.sum(sq_displacement(data))/len(data)

Below we see the results for distributions of MSDs obtained from 1000 simulated experiments in which N=1000 displacements are sampled (to generate this data the displacement array for a particle that took 1,000,000 steps  was split into 1000 arrays). The value of D is chosen with D = $2.19*10^{-10} m^2/s$ and $\tau = 1$s such that the true value of the MSD should be 

$$\langle\Delta x^2 \rangle = 6*D*\tau = 1.31*10^{-9}m^2$$

In [ ]:
test_data = np.split(data,1000)
msd_array = np.array(list(map(MSD,test_data)))
plt.hist(msd_array)
np.median(msd_array)
plt.ylabel('counts')
plt.xlabel('$\langle \Delta x^2 \\rangle$ (m$^2)$')

The standard error of the of sample variance goes as 1/$\sqrt{N}$, but to calculate the true standard error on the sample variance we use the FP algorithm.

# TODO:

Verify that the calculations for the variance of the mean and that uncertainty on the variance is correct.

In [ ]:
msd_array = sq_displacement(data)
new_length = len(msd_array)

output_var = []
var_uncertainty = []

for i in range(0,10):
    #length of the next array should be 2 times smalller (2^(i+1) times smaller overall)
    new_length = len(msd_array)/2
    new_array = np.zeros(new_length)
    #iterate over each element of the new array
    for i in range(0,len(new_array)):
        #Each element is given by the average of the corresponding 2 elements in the previous array
        new_array[i] = (msd_array[2*i-1]+msd_array[2*i])/2
    output_var.append(np.var(new_array)/(len(new_array)-1))
    var_uncertainty.append(np.var(new_array)/(len(new_array)-1)*np.sqrt(2/(len(new_array)-1)))
    msd_array = new_array

In [ ]:
b = np.arange(0,len(output_var))
plt.errorbar(b,output_var, yerr = var_uncertainty, fmt='o')
plt.ylabel('Output Variance of the Mean')
plt.xlabel('Nuber of Block Transformations')

We can see that from the FP method, we should take the average of points 3 through 5 to find the variance on the sample mean.

In [ ]:
print(np.sqrt(np.mean(output_var[2:7])))

We see that from the FP method we obtain a 68% confidence interval (1-sigma) of $\pm 1.8*10^{-11}$

Compare this to the 68% credibility interval by MCMC of $~ \pm 9*10^{-12}$

## CGW Method

This method is alluded to in Jerome Fung's Thesis, and is one of the primary ways of estimating the diffusion coefficient in co The idea is to take a set of MSDs and then fit a line of the form 2D$\tau$ to the plot of MSD as a function of lag time. The relative standard error of the variance, $\sigma_{var}/var$ can be computed by:

$$\frac{\sigma_{var}}{var}=\sqrt{\frac{2}{N_{ind}-1}}$$

where 

$$N_{ind, CGW} = 2\frac{N-n}{n}$$